# 6.4 循环神经网络的从零开始实现

In [19]:
import inspect
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print(device)

1.9.0
cpu


In [20]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

## 6.4.1 one-hot向量

In [21]:
# inspect.getsource(torch._C._nn.one_hot)

In [22]:
def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res
    
x = torch.tensor([0, 2])
y = one_hot(x, vocab_size)
y, x.shape, y.shape

(tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.]]),
 torch.Size([2]),
 torch.Size([2, 1027]))

In [23]:
# F.one_hot(x, vocab_size)

In [24]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1027])


## 6.4.2 初始化模型参数

In [25]:
# num_hiddens属于超参数，需要慢慢调，过小过大都不好
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(
            np.random.normal(0, 0.01, size=shape),
            device=device,
            dtype=torch.float32
        )
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(
        num_hiddens, device=device, requires_grad=True))

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(
        num_outputs, device=device, requires_grad=True))

    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

will use cpu


## 6.4.3 定义模型

In [26]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [27]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        #  这里之所以用tanh作为激活函数，是因为其输出依原点对称
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [28]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
state[0].shape

torch.Size([2, 256])

In [29]:
inputs = to_onehot(X.to(device), vocab_size)
inputs[0].shape

torch.Size([2, 1027])

In [30]:
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


## 6.4.4 定义预测函数

In [31]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
#     print(output)
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
#         print(Y[0].shape)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
#         print(output)
    return ''.join([idx_to_char[i] for i in output])

In [32]:
# 我们这里利用还没有进行过training的rnn来做一下预测， 可以发现预测的结果并不好
predict_rnn('分开不要', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开不要物二抬相壁隔注许照穿'

## 6.4.5 裁剪梯度

In [33]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

## 6.4.6 困惑度
## 6.4.7 定义模型训练函数

In [34]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
            
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

## 6.4.8 训练模型并创作歌词

In [35]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [36]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 68.859614, time 0.77 sec
 - 分开 我想想你 你我的外  有我的见 在人的可 在人了 别怪我 说你的让我疯狂的可爱女人 坏坏的让我疯狂
 - 不分开  我想要这 我不了  什么我 别怪我 别怪我 一颗四颗 在人的双  有我的见 在人的让 在人的可爱
epoch 100, perplexity 10.483102, time 0.80 sec
 - 分开 一只两 一步四步三步四步 连成线背著背默默一个的愿 在小方的星色果听的见 它在灌 一步两步三步四步
 - 不分开力 我不要再想 我不 我不 我不要再想 我不 我不 我不要再想 我不 我不 我不要再想 我不 我不 
epoch 150, perplexity 2.928586, time 0.87 sec
 - 分开 一只两停留 用话躺停留的 为什么我女朋友场外加油 你却还让我出糗 从小这耳濡目染 什么刀枪武棍  
 - 不分开吗 我后能爸 你打我妈 这样跟  对我有没 我跟你这节 再一定实 温家了空演 白色蜡烛 温暖了空屋 
epoch 200, perplexity 1.608186, time 0.85 sec
 - 分开 一只两步三步四步 连成让背著背 游荡法蓝 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双
 - 不分开扫简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 我想大声宣布 对
epoch 250, perplexity 1.322037, time 0.76 sec
 - 分开球默愿想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 一梭在忆的画面的钟 从反方向开始移
 - 不分开扫简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 开不了口 周杰伦


In [37]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 65.643185, time 0.74 sec
 - 分开 我想你的可爱女人 坏坏我 我给我的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏
 - 不分开 我想就这 温使了人 如果的可 在果的让 如果的可爱女人 我想想你 温小的让我疯狂的可爱女人 坏坏的
epoch 100, perplexity 7.454730, time 0.80 sec
 - 分开 我有能这样 别不的角丽 你的完美主义 太彻底 让我连恨都难以 我不要你的微笑每天都能不着  我知道
 - 不分开柳 你已经能 我想不能 我 再真了骑听  思悔了霜坦堡 一直都我 再堡之碗的溪边 我默店够二 三两银
epoch 150, perplexity 2.055571, time 0.71 sec
 - 分开 我想要这样布着 但一枝杨柳勉刚 去不去童南故事  有教堂有城堡 每天忙碌地的寻找 到底什么我想要 
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 200, perplexity 1.311746, time 0.74 sec
 - 分开 她候我 谁怪神枪我 泪有没容身里我 妈不球我满腔的怒火 我想揍你已经很久 别想躲 说你眼睛看着我 
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 250, perplexity 1.168402, time 0.72 sec
 - 分开 问候不美 我想一难是我听错 誓言太沉重泪被纵容 脸上汹涌失控 穿梭时间 不安跳动 染红夜空 过去种
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
